In [ ]:
import os
import tempfile
import unittest
import filecmp
import sbol3
import paml
import tyto
import uml
from paml.execution_engine import ExecutionEngine

%load_ext autoreload
%autoreload 2

In [ ]:
#############################################
# set up the document
print('Setting up document')
doc = sbol3.Document()
sbol3.set_namespace('https://bbn.com/scratch/')

#############################################
# Import the primitive libraries
print('Importing libraries')
paml.import_library('liquid_handling')
print('... Imported liquid handling')
paml.import_library('plate_handling')
print('... Imported plate handling')
paml.import_library('spectrophotometry')
print('... Imported spectrophotometry')
paml.import_library('sample_arrays')
print('... Imported sample arrays')

In [ ]:
# Print descriptions of the primitives avaiable in each library
dashes = "-" * 80
print(dashes)
primitives = {}
for lib, lib_doc in paml.loaded_libraries.items():
    print(f"{dashes}\nlibrary: {lib}")
    for primitive in lib_doc.objects:
        primitives[primitive.identity] = primitive
        print(primitive)
    print(dashes)

In [ ]:
# create the materials to be provisioned
ddh2o = sbol3.Component('ddH2O', 'https://identifiers.org/pubchem.substance:24901740')
ddh2o.name = 'Water, sterile-filtered, BioReagent, suitable for cell culture'  # TODO get via tyto
doc.add(ddh2o)

ludox = sbol3.Component('LUDOX', 'https://identifiers.org/pubchem.substance:24866361')
ludox.name = 'LUDOX(R) CL-X colloidal silica, 45 wt. % suspension in H2O'
doc.add(ludox)

# add an optional parameter for specifying the wavelength
wavelength_param = protocol.input_value('wavelength', sbol3.OM_MEASURE, optional=True,
                                              default_value=sbol3.Measure(600, tyto.OM.nanometer))

In [ ]:
# actual steps of the protocol
# get a plate
plate = protocol.primitive_step('EmptyContainer', specification=tyto.NCIT.get_uri_by_term('Microplate'))  # replace with container ontology

protocol.to_dot()

In [ ]:
# put water in selected wells
c_ddh2o = protocol.primitive_step('PlateCoordinates', source=plate.output_pin('samples'), coordinates='A1:D1')
provision_ddh2o = protocol.primitive_step('Provision', resource=ddh2o, destination=c_ddh2o.output_pin('samples'),
                        amount=sbol3.Measure(100, tyto.OM.microliter))

protocol.to_dot()

In [ ]:
# put ludox in selected wells
c_ludox = protocol.primitive_step('PlateCoordinates', source=plate.output_pin('samples'), coordinates='A2:D2')
provision_ludox = protocol.primitive_step('Provision', resource=ludox, destination=c_ludox.output_pin('samples'),
                        amount=sbol3.Measure(100, tyto.OM.microliter))

protocol.to_dot()

In [ ]:
# measure the absorbance
c_measure = protocol.primitive_step('PlateCoordinates', source=plate.output_pin('samples'), coordinates='A1:D2')
measure = protocol.primitive_step('MeasureAbsorbance', samples=c_measure.output_pin('samples'))
protocol.to_dot()

In [ ]:
protocol.use_value(wavelength_param, measure.input_pin('wavelength'))
output = protocol.designate_output('absorbance', sbol3.OM_MEASURE,
                                           measure.output_pin('measurements'))
protocol.order(protocol.get_last_step(), output)
protocol.to_dot()

In [ ]:
protocol.to_dot()

In [ ]:
#to_remove = [x for x in doc.objects if x.identity == "https://bbn.com/scratch/ludox_protocol_constraints"]
#doc.objects.remove(next(iter(to_remove)))

In [ ]:
import paml_time as pamlt

# protocol starts at time 0
protocol_start_time = pamlt.startTime(protocol, 0, units=tyto.OM.hour)
provision_ludox_duration = pamlt.duration(provision_ludox, 60, units=tyto.OM.second)
provision_ddh2o_duration = pamlt.duration(provision_ddh2o, 60, units=tyto.OM.second)
execute_measurement_duration = pamlt.duration(measure, 2, units=tyto.OM.minute)
ddh2o_before_ludox_constraint = pamlt.precedes(provision_ddh2o, [0, 60], provision_ludox, units=tyto.OM.second)

time_constraints = pamlt.TimeConstraints("ludox_protocol_constraints",
    constraints=[pamlt.And([
        protocol_start_time,
        provision_ludox_duration,
        provision_ddh2o_duration,
        execute_measurement_duration,
        ddh2o_before_ludox_constraint
        ])],
    protocols = [protocol]
)

doc.add(time_constraints)

In [ ]:
# Compute a Schedule for the protocol and get the constraint graph
schedule, graph = pc.check_doc(doc)
schedule.plot()

In [ ]:
# Get minimum duration for the protocol (in seconds)
min_protocol_durations = graph.get_minimum_duration()
min_str = "\n".join([f"The minimum duration of {protocol} is {result['duration']}s " \
                     for protocol, result in min_protocol_durations.items()])
print(min_str)

In [ ]:
# Simulate Execution of the Protocol

agent = sbol3.Agent("test_agent")
ee = ExecutionEngine()
parameter_values = [
    paml.ParameterValue(parameter=protocol.get_input("wavelength"), 
                        value=sbol3.Measure(100, tyto.OM.nanometer))
]
execution = ee.execute(protocol, agent, id="test_execution", parameter_values=parameter_values)
execution.to_dot()

In [ ]:
# Compile to autoprotocol and opentrons (connect to simulator?)

In [ ]:
# Spoof execution of protocol with real times and data